In [1]:
import numpy as np
import cv2
import os
import math
import time
import random as random

Implementacion del detector de fondo naive utilizando la mediana como estimador.

Comienzo definiendo las funciones init_N_frames para seleccionar N frames aleatorios dentro del video para calcular la mediana. 
Luego la funcion calculate_median para calcular la mediana de los frames recibidos y asi obtener el estimador del fondo. Aplico desenfoque gaussiano para suavizar la imagen y trabajo en escala de grises. 

In [44]:
def init_N_frames(cap,N):
    frames=[]
    total_frames=cap.get(cv2.CAP_PROP_FRAME_COUNT)
    frames_id=np.random.uniform(size=N)
    for f in frames_id:
        cap.set(cv2.CAP_PROP_POS_FRAMES,f)
        ret,frame=cap.read()
        frames.append(frame)
    return(frames)

In [32]:
def calculate_median(frames,N): 
        mediana = np.median(np.array(random.choices(frames, k=N)), axis=0).astype(dtype=np.uint8)
        mediana = cv2.normalize(mediana, mediana, 0, 255, cv2.NORM_MINMAX).astype('uint8') 
        mediana = cv2.GaussianBlur(mediana, (3, 3), 0)  
        mediana_gray = cv2.cvtColor(mediana, cv2.COLOR_BGR2GRAY)
        return mediana_gray

In [56]:
def naive_back_detection(cap,N,t):#recibo video, Nro de frames para calcular la mediana, tiempo para recalcular el fondo
    start_time = time.time()
    ret,frame=cap.read()
    fps = math.ceil(cap.get(cv2.CAP_PROP_FPS)) 
    w = int(cap.get(3)) 
    h= int(cap.get(4))
    salida = cv2.VideoWriter('naive_back_det_resul_6.mp4', cv2.VideoWriter_fourcc(*'XVID'), fps, (w,h))
    #inicializacion de variables 
    frames=[]
    rec_frames=[]
    cont_frames=0
    actualizar_back=t*fps#calculo el nro de frames para actualizar el fondo
    frames=init_N_frames(cap,N)#obtengo N frames aleatorios para calcular la mediana
    mediana_gray=calculate_median(frames,N) #calculo la mediana   
    while True:
        ret,frame=cap.read()
        cont_frames+=1 
        if not ret:
            break
        rec_frames.append(frame)#voy guardando los frames para recalcular luego el nuevo fondo
        if cont_frames==actualizar_back:
            cv2.putText(frame, "Recalculando fondo" , (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),2)
            mediana_gray=calculate_median(rec_frames,N)#vuelvo a calcular la mediana 
            cont_frames=0
        frame_gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        frame_gray= cv2.GaussianBlur(frame_gray, (3, 3), 0) 
        diff=cv2.absdiff(frame_gray,mediana_gray) #hago la resta para que solo queden los objetos 
        th,diff_th=cv2.threshold(diff,50,255,cv2.THRESH_BINARY)#binarizo para obtener mejores resultados 
        cnts = cv2.findContours(diff_th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]#busco los contornos en la imagen binarizada
        for cnt in cnts:
                if cv2.contourArea(cnt) > 500:#filtro para descartar algunos objetos que no sean los de interes
                    x, y, w, h = cv2.boundingRect(cnt)
                    cv2.rectangle(frame, (x,y), (x+w, y+h),(0,255,0), 2)
        salida.write(frame)
    salida.release()
    end_time=time.time()        
    cap.release()
    print("Tiempo de procesamiento:",end_time-start_time)

A continuacion utilizo una funcion de openCV que aplica un metodo basado en mezcla de gaussianas para comparar los resultados. 

In [27]:
def MOG2(cap):
    start_time = time.time()
    fgbg = cv2.createBackgroundSubtractorMOG2()
    fps = math.ceil(cap.get(cv2.CAP_PROP_FPS)) 
    w = int(cap.get(3)) 
    h= int(cap.get(4))
    salida = cv2.VideoWriter('MOG2.mp4', cv2.VideoWriter_fourcc(*'XVID'), fps, (w,h))
    while True:
        ret, frame = cap.read()
        if not ret:
            break      
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray=cv2.GaussianBlur(gray, (3, 3), 0)       
        fgmask = fgbg.apply(gray)
        _,th=cv2.threshold(fgmask,150,255,cv2.THRESH_BINARY)
        cnts = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
        for cnt in cnts:
            if cv2.contourArea(cnt) > 300:
                x, y, w, h = cv2.boundingRect(cnt)
                cv2.rectangle(frame, (x,y), (x+w, y+h),(0,255,0), 2)
        salida.write(frame)   
    end_time=time.time()        
    salida.release()
    cap.release()
    cv2.destroyAllWindows()
    print("Tiempo de procesamiento:",end_time-start_time)
    

Pruebo ambos metodos con distintos parametros y comparo resultados

In [53]:
cap = cv2.VideoCapture('vtest.avi')
naive_back_detection(cap,25,5)

Tiempo de procesamiento: 9.911282777786255


In [55]:
cap = cv2.VideoCapture('vtest.avi')
naive_back_detection(cap,10,1)

Tiempo de procesamiento: 14.6244957447052


In [29]:
cap = cv2.VideoCapture('vtest.avi')
MOG2(cap)

Tiempo de procesamiento: 7.9368321895599365


Hice varias pruebas para ver la variacion en el tiempo del procesamiento y pude comprobar que menor es el tiempo que seteamos para recalcular el fondo, mas tarda ya que debe realizar las operaciones mayor cantidad de veces. Mas dilatamos este tiempo, mas velocidad obtenemos y mas parecido al resultado utilizando MOG2 de opencv. 

Hay algunos frames donde queda detectando cuadros que en frames anteriores hubo movimiento. Estos desaparecen en cuanto se recalcula el fondo, por lo cual es algo muy importante de tener en cuenta y lo mejor seria recalcular en cortos lapsos de tiempo, esto si sacrificando la velocidad del procesamiento como explicaba en el item anterior. 

Otro detalle que tiene este metodo es que cuando varias personas estan cruzandose, o estan cerca, las toma como un solo cuadro. Probe con distintas operaciones morfologicas para poder evitar esto pero no lograba buenos resultados, separaba quizas las personas pero tambien perdia partes del cuerpo. Por lo tanto decidi tomarlo como una sola figura. Esto se debe tener en cuenta dependiendo la aplicacion para la que necesitemos el algoritmo.